# Evaluation MetaCAT - BiLSTM

In [1]:
from tqdm.notebook import tqdm
from tokenizers import ByteLevelBPETokenizer
import pandas as pd
from gensim.models import Word2Vec
import json
import numpy as np
import os

from medcat.cat import CAT
from medcat.vocab import Vocab
from medcat.cdb import CDB
from medcat.config import Config
from medcat.meta_cat import MetaCAT
from medcat.preprocessing.tokenizers import TokenizerWrapperBPE

/Users/stan3/Data/MedCAT/medcat/cat.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Input
data_dir = os.path.join('..', 'data')
cdb_file = os.path.join(data_dir, 'cdb.dat')
vocab_file = os.path.join(data_dir, 'vocab.dat')

# Output
output_dir = 'output'
tokenizer_name = 'emc_dcc'

## Load tokenizer

In [3]:
# Create, train and save the tokenizer
mc_negation = MetaCAT(save_dir=output_dir, device='cpu')
mc_negation.load(tokenizer_name=tokenizer_name)

## Example usage

In [4]:
# Load the cdb and vocab 
config = Config()

vocab = Vocab.load(vocab_file)
cdb = CDB.load(cdb_file)

# Create MedCAT pipeline
cat = CAT(cdb=cdb, vocab=vocab, config=config, meta_cats=[mc_negation])

In [5]:
# Test on DL1114 from DCC with negation
text = 'Echo- en rontgenonderzoek van de heup toont geen evidente heupdysplasie.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: heup
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9943742, 'name': 'Negation'}}


Entity: heupdysplasie
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9997235, 'name': 'Negation'}}




In [6]:
# Test on DL1114 from DCC without negation
text = 'Echo- en rontgenonderzoek van de heup toont evidente heupdysplasie.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: heup
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.99893314, 'name': 'Negation'}}


Entity: heupdysplasie
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.99205214, 'name': 'Negation'}}




## Additional testing

In [7]:
# Test on DL1112 from DCC
text = 'Patient kan zich geen trauma herinneren.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

# Trauma is not identified as medical concept.

In [8]:
# Test on NTvG article
# https://www.ntvg.nl/artikelen/acute-buik-op-basis-van-een-wandelende-milt
text = '1 maand na de operatie had patiënte geen buikpijn meer en was zij goed hersteld.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")
    
# The negation was missed.

Entity: operatie
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.999671, 'name': 'Negation'}}


Entity: buikpijn
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.94574124, 'name': 'Negation'}}




In [9]:
# Test on NTvG article
# https://www.ntvg.nl/artikelen/een-bezoar-bij-een-vrouw-met-clomipramine-intoxicatie
text = 'Er waren geen tekenen van darmobstructie te zien.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

# Correct identification of negation

Entity: darmobstructie
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9341217, 'name': 'Negation'}}


Entity: zien
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.8449472, 'name': 'Negation'}}




In [10]:
# Test on NTvG article
# https://www.ntvg.nl/artikelen/nieuws/vaker-ziek-na-acute-fase-covid-19
text = 'Alle patiënten werden gematcht met een controlegroep bij wie geen SARS-CoV-2-infectie was geregistreerd.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")
    
# Negation on SARS-CoV was missed

Entity: patiënten
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9957676, 'name': 'Negation'}}


Entity: controlegroep
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.99965554, 'name': 'Negation'}}


Entity: SARS-CoV
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.59226084, 'name': 'Negation'}}


Entity: infectie
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9611355, 'name': 'Negation'}}




## Evaluate MetaCat on subsets of the data
The ContextD paper calculates precision, recall and F1-score on subsets of the data. In this section we calculate the same scores with the just created model. Note that this results in a calculation on a set of data that was included during the training phase. For proper score calculations, we will do cross validation at a later stage.

In [11]:
json_file_DL = os.path.join(data_dir, 'emc-dcc_ann_DL.json')
json_file_GP = os.path.join(data_dir, 'emc-dcc_ann_GP.json')
json_file_RD = os.path.join(data_dir, 'emc-dcc_ann_RD.json')
json_file_SP = os.path.join(data_dir, 'emc-dcc_ann_SP.json')

In [12]:
mc_negation.eval(json_file_RD)

**************************************************  Test
              precision    recall  f1-score   support

           0       0.94      0.81      0.87       595
           1       0.96      0.99      0.98      3088

    accuracy                           0.96      3683
   macro avg       0.95      0.90      0.92      3683
weighted avg       0.96      0.96      0.96      3683

Test Loss:  0.12919218332280177








{'0': {'precision': 0.9412915851272016,
  'recall': 0.8084033613445378,
  'f1-score': 0.8698010849909584,
  'support': 595},
 '1': {'precision': 0.9640605296343001,
  'recall': 0.9902849740932642,
  'f1-score': 0.9769968051118211,
  'support': 3088},
 'accuracy': 0.9609014390442574,
 'macro avg': {'precision': 0.9526760573807509,
  'recall': 0.899344167718901,
  'f1-score': 0.9233989450513898,
  'support': 3683},
 'weighted avg': {'precision': 0.9603821364815106,
  'recall': 0.9609014390442574,
  'f1-score': 0.9596790061783664,
  'support': 3683}}

In [13]:
mc_negation.eval(json_file_SP)

**************************************************  Test
              precision    recall  f1-score   support

           0       0.91      0.69      0.78       416
           1       0.95      0.99      0.97      2309

    accuracy                           0.94      2725
   macro avg       0.93      0.84      0.87      2725
weighted avg       0.94      0.94      0.94      2725

Test Loss:  0.23037438435546523








{'0': {'precision': 0.9076433121019108,
  'recall': 0.6850961538461539,
  'f1-score': 0.7808219178082193,
  'support': 416},
 '1': {'precision': 0.9456656988801327,
  'recall': 0.9874404504114335,
  'f1-score': 0.9661016949152542,
  'support': 2309},
 'accuracy': 0.9412844036697248,
 'macro avg': {'precision': 0.9266545054910218,
  'recall': 0.8362683021287937,
  'f1-score': 0.8734618063617368,
  'support': 2725},
 'weighted avg': {'precision': 0.9398611803848151,
  'recall': 0.9412844036697248,
  'f1-score': 0.9378167821532262,
  'support': 2725}}

In [14]:
mc_negation.eval(json_file_DL)

**************************************************  Test
              precision    recall  f1-score   support

           0       0.97      0.88      0.92       379
           1       0.98      1.00      0.99      2417

    accuracy                           0.98      2796
   macro avg       0.97      0.94      0.95      2796
weighted avg       0.98      0.98      0.98      2796

Test Loss:  0.07555809380885746








{'0': {'precision': 0.9652173913043478,
  'recall': 0.8786279683377308,
  'f1-score': 0.9198895027624309,
  'support': 379},
 '1': {'precision': 0.9812321501427989,
  'recall': 0.9950351675630947,
  'f1-score': 0.9880854560394412,
  'support': 2417},
 'accuracy': 0.9792560801144492,
 'macro avg': {'precision': 0.9732247707235733,
  'recall': 0.9368315679504128,
  'f1-score': 0.953987479400936,
  'support': 2796},
 'weighted avg': {'precision': 0.9790613369812207,
  'recall': 0.9792560801144492,
  'f1-score': 0.9788414409135517,
  'support': 2796}}

In [15]:
mc_negation.eval(json_file_GP)

**************************************************  Test
              precision    recall  f1-score   support

           0       0.88      0.72      0.79       383
           1       0.96      0.99      0.98      3024

    accuracy                           0.96      3407
   macro avg       0.92      0.85      0.88      3407
weighted avg       0.96      0.96      0.95      3407

Test Loss:  0.1529528521001339








{'0': {'precision': 0.8782051282051282,
  'recall': 0.7154046997389034,
  'f1-score': 0.7884892086330936,
  'support': 383},
 '1': {'precision': 0.9647819063004847,
  'recall': 0.9874338624338624,
  'f1-score': 0.9759764667429318,
  'support': 3024},
 'accuracy': 0.956853536835926,
 'macro avg': {'precision': 0.9214935172528065,
  'recall': 0.851419281086383,
  'f1-score': 0.8822328376880126,
  'support': 3407},
 'weighted avg': {'precision': 0.9550493245539271,
  'recall': 0.956853536835926,
  'f1-score': 0.9548999713346348,
  'support': 3407}}